In [33]:
import pysolr, os
import spacy

nlp = spacy.load('en_core_web_sm')

PMC_URL = 'http://' + os.environ['SOLR_HOST'] + ':8983/solr/pmc'
MESH_URL = 'http://' + os.environ['SOLR_HOST'] + ':8983/solr/mesh3'

pmc_solr = pysolr.Solr(PMC_URL, always_commit=False)
mesh_solr = pysolr.Solr(MESH_URL, always_commit=False)

# Do a health check.
mesh_solr.ping()
pmc_solr.ping()

'{\n  "responseHeader":{\n    "zkConnected":null,\n    "status":0,\n    "QTime":0,\n    "params":{\n      "q":"{!lucene}*:*",\n      "distrib":"false",\n      "df":"_text_",\n      "rows":"10",\n      "echoParams":"all"}},\n  "status":"OK"}\n'

In [34]:
def get_n_grams(doc):
    i = 0
    n_grams = []
    for i in range(i, len(doc)):
        n_gram = []
        n_gram.append(doc[i].text)

        if (i<len(doc)-1):
            n_gram.append(doc[i+1].text)
        if (i<len(doc)-2):
            n_gram.append(doc[i+2].text)    

        n_grams.append(n_gram)
    return n_grams
     

In [35]:
def search(n_gram):
    separator = ' '
    text_value = separator.join(n_gram)
    query = '{}:\"{}\"'.format('ConceptName', text_value)
    results = mesh_solr.search(query)
    if (len(results) != 0):
        for r in results:
            return (r['ConceptName'], len(n_gram))
    elif len(n_gram)-1>0: 
        return search(n_gram[:len(n_gram)-1])
    else: return ('', len(n_gram)-1)

In [41]:
def get_labels(n_grams):
    n = 0
    word_index = 0
    labels = ['O' for i in range(len(n_grams))]
    for n_gram in n_grams:
        if n > 1:
            word_index = word_index + 1
            n= n-1
            continue

        concept, n = search(n_gram)
        if (concept):
            for i in range(n):
                labels[word_index+i] = concept

        word_index = word_index + 1
    return labels

In [37]:
sentences = []
i=0
nrows = 1000
numFound=0
while i*nrows <= 2000:
    results = solr.search('*:*', rows=1000, start=i*nrows)
    
#     print(results.raw_response['response'])
#     numFound = results.raw_response['response']['numFound']
    i = i+1
    
    for result in results:
        sentence = ''
        if 'abstract' in result:
            for abstract in result['abstract']:
                sentence = sentence + abstract + ' '
            sentences.append(sentence)
            

In [42]:
for sentence in sentences:
    doc = nlp(sentence)
    n_grams =  get_n_grams(doc)
    labels = get_labels(n_grams)
    print(labels)
    break

['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'Epidermal Growth Factor', 'Epidermal Growth Factor', 'Epidermal Growth Factor', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'Antibodies', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'Cell Line', 'Cell Line', 'Cell Line', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'Cheek', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'Cells', 'Cells', 'Fluorescence', 'O', 'O', 'O', 'Cells', 'O', 'O', 'O'

In [43]:
for label, d in zip (labels, doc):
    print(label, d)

O Objectives
O :
O :
O The
O purpose
O of
O this
O study
O was
O to
O investigate
O in
O -
O vivo
O visible
O imaging
O of
O oral
O squamous
O cell
O carcinoma
O (
O OSCC
O )
O by
O targeting
Epidermal Growth Factor epidermal
Epidermal Growth Factor growth
Epidermal Growth Factor factor
O receptor
O (
O EGFR
O )
O with
O near
O -
O infrared
O quantum
O dots
O .
O Materials
O and
O methods
O :
O :
O Quantum
O dots
O with
O an
O emission
O wavelength
O of
O 800
O nm
O (
O QD800
O )
O were
O conjugated
O to
O monoclonal
Antibodies antibodies
O against
O EGFR
O ,
O resulting
O in
O the
O probe
O designated
O as
O QD800-EGFR
O Ab
O .
O OSCC
Cell Line cell
Cell Line line
Cell Line (
O BcaCD885
O )
O expressing
O high
O levels
O of
O EGFR
O was
O transplanted
O subcutaneously
O into
O nude
O mice
O cheeks
O to
O develop
O an
O OSCC
O animal
O model
O .
O QD800-EGFR
O Ab
O containing
O 100
O pmol
O equivalent
O of
O QD800
O was
O intravenously
O injected
O into
O the
O animal
O model
O ,
O and